"This is a transnational data set which contains all the transactions occurring between 2010-12-01 and 2011-12-09 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers."

In [1]:
import numpy as np
import pandas as pd
import time, warnings
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

%matplotlib inline
warnings.filterwarnings("ignore")

In [6]:
#read the csv
df = pd.read_csv('uk_retailer_transactions.csv', encoding="ISO-8859-1", dtype={'CustomerID': str,'InvoiceID': str})
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [7]:
#data cleansing
#keep all data that are from the United Kingdom and remove empty rows and canceled orders
df_copy = df[df['Country']=='United Kingdom']
df_copy = df_copy[df_copy['Quantity']>0]
df_copy.dropna(subset=['CustomerID'],how='all',inplace=True)
#restrict the data to one full year because it's better to use a metric per Months or Years in RFM
# df_copy = df_copy[retail_uk['InvoiceDate']>= "2010-12-09"]
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354345 entries, 0 to 541893
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    354345 non-null  object 
 1   StockCode    354345 non-null  object 
 2   Description  354345 non-null  object 
 3   Quantity     354345 non-null  int64  
 4   InvoiceDate  354345 non-null  object 
 5   UnitPrice    354345 non-null  float64
 6   CustomerID   354345 non-null  object 
 7   Country      354345 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 24.3+ MB


RFM Analysis
RFM analysis is a marketing technique used to quantitatively rank and group customers based on the recency, frequency and monetary total of their recent transactions to identify the best customers and perform targeted marketing campaigns.

RECENCY (R): Days since last purchase
FREQUENCY (F): Total number of purchases
MONETARY VALUE (M): Total money this customer spent.

Recency

In [9]:
#add a data column for each transaction
df_copy['date'] = pd.DatetimeIndex(df_copy['InvoiceDate']).date
df_copy.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01


In [19]:
#create a dataframe to calculate recency later
recency_df = df_copy.groupby(by='CustomerID', as_index=False)['date'].max()
recency_df.columns = ['CustomerID','LastPurshaceDate']
recency_df.head()

,CustomerID,LastPurshaceDate
0,12346,2011-01-18
1,12747,2011-12-07
2,12748,2011-12-09
3,12749,2011-12-06
4,12820,2011-12-06


In [20]:
#calculate recency
last_day = dt.date(2011,12,9)
recency_df['Recency'] = recency_df['LastPurshaceDate'].apply(lambda x: (last_day - x).days)
recency_df.drop('LastPurshaceDate',axis=1,inplace=True)
recency_df.head()

,CustomerID,Recency
0,12346,325
1,12747,2
2,12748,0
3,12749,3
4,12820,3


Frequency

In [21]:
frequency_copy = df_copy
frequency_copy.drop_duplicates(subset=['InvoiceNo', 'CustomerID'], keep="first", inplace=True)
#calculate frequency of purchases
frequency_df = frequency_copy.groupby(by=['CustomerID'], as_index=False)['InvoiceNo'].count()
frequency_df.columns = ['CustomerID','Frequency']
frequency_df.head()

,CustomerID,Frequency
0,12346,1
1,12747,11
2,12748,210
3,12749,5
4,12820,4


Monetary

In [22]:
#create a column for total cost of transaction
df_copy['TotalCost'] = df_copy['Quantity'] * df_copy['UnitPrice']

In [23]:
df_copy.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,TotalCost
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom,2010-12-01,15.30
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850,United Kingdom,2010-12-01,11.10
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047,United Kingdom,2010-12-01,54.08
21,536368,22960,JAM MAKING SET WITH JARS,6,12/1/2010 8:34,4.25,13047,United Kingdom,2010-12-01,25.50
25,536369,21756,BATH BUILDING BLOCK WORD,3,12/1/2010 8:35,5.95,13047,United Kingdom,2010-12-01,17.85


In [24]:
monetary_df = df_copy.groupby(by='CustomerID',as_index=False).agg({'TotalCost': 'sum'})
monetary_df.columns = ['CustomerID','Monetary']
monetary_df.head()

,CustomerID,Monetary
0,12346,77183.60
1,12747,689.49
2,12748,3841.31
3,12749,98.35
4,12820,58.20


In [25]:
#merge the RFM table
temp_df = recency_df.merge(frequency_df,on='CustomerID')
rfm_df = temp_df.merge(monetary_df,on='CustomerID')
rfm_df.set_index('CustomerID',inplace=True)
rfm_df.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346,325,1,77183.60
12747,2,11,689.49
12748,0,210,3841.31
12749,3,5,98.35
12820,3,4,58.20


In [28]:
#seperate customers into 4 quantiles and create a dictionary
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
print(quantiles)
quantiles.to_dict()

      Recency  Frequency  Monetary
0.25     17.0        1.0      17.7
0.50     50.0        2.0      45.4
0.75    142.0        5.0     124.5


{'Recency': {0.25: 17.0, 0.5: 50.0, 0.75: 142.0},
 'Frequency': {0.25: 1.0, 0.5: 2.0, 0.75: 5.0},
 'Monetary': {0.25: 17.700000000000003, 0.5: 45.4, 0.75: 124.50000000000001}}

In [41]:
#now we write two functions to score customers from 1-4 for each RFM variable (1 being the lowest and 4 being the highest)
def RScore(variable_value, variable_name, quartile_dict):
    if variable_value <= quartile_dict[variable_name][0.25]:
        return 4
    elif variable_value <= quartile_dict[variable_name][0.50]:
        return 3
    elif variable_value <= quartile_dict[variable_name][0.75]: 
        return 2
    else:
        return 1

def FMScore(variable_value, variable_name, quartile_dict):
    if variable_value <= quartile_dict[variable_name][0.25]:
        return 1
    elif variable_value <= quartile_dict[variable_name][0.50]:
        return 2
    elif variable_value <= quartile_dict[variable_name][0.75]: 
        return 3
    else:
        return 4

In [42]:
#now score all customers
score_table = rfm_df
score_table['R_Quartile'] = score_table['Recency'].apply(RScore, args=('Recency',quantiles,))
score_table['F_Quartile'] = score_table['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
score_table['M_Quartile'] = score_table['Monetary'].apply(FMScore, args=('Monetary',quantiles,))
score_table.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile
CustomerID,,,,,,
12346,325,1,77183.60,1,1,4
12747,2,11,689.49,4,4,4
12748,0,210,3841.31,4,4,4
12749,3,5,98.35,4,3,3
12820,3,4,58.20,4,3,3


In [43]:
#now we combine all the scores to categorize different customer groups
score_table['RFMScore'] = score_table.R_Quartile.map(str) + score_table.F_Quartile.map(str) + score_table.M_Quartile.map(str)
score_table.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
12346,325,1,77183.60,1,1,4,114
12747,2,11,689.49,4,4,4,444
12748,0,210,3841.31,4,4,4,444
12749,3,5,98.35,4,3,3,433
12820,3,4,58.20,4,3,3,433


In [58]:
#I choose to put customers into 6 groups
print("Number of Best Customers: ",len(score_table[score_table['RFMScore']=='444']))

print('Number of Loyal Customers: ',len(score_table[(score_table['F_Quartile']==4) &
                                                         (score_table['RFMScore']!='444')]))

print("Number of Big Spenders: ",len(score_table[(score_table['M_Quartile']==4) &
                                                     (score_table['RFMScore']!='444')]))

print('Number of Inactive Customers: ', len(score_table[(score_table['R_Quartile']==2) & (score_table['F_Quartile']<4)]))

print('Number of Lost Customers: ',len(score_table[score_table['R_Quartile']==1]))

print('Number of Small Buyer Customers: ',len(score_table[(score_table['R_Quartile']>2) & (score_table['F_Quartile']>2) &
                                                              (score_table['M_Quartile']==1)]))

Number of Best Customers:  370
Number of Loyal Customers:  421
Number of Big Spenders:  610
Number of Inactive Customers:  880
Number of Lost Customers:  979
Number of Small Buyer Customers:  47


For best customers,  provide them with VIP services and give them privileges like they can add new products early and have more discounts

Connect with the loyal customers and make them a priority in case studies. Create a loyal level program and ask to refer a friend.

Big spenders are people who spend a lot but do not visit the website frequently. Offer them time-bound deals to create urgency and make their websites seamless and compelling.

Find out why lost customers left, and reach out to them as soon as possible to make things right for them.

Welcome inactive users back to the websites and provide them with extended offers.

For small buyers, email them a page of coupons and discounts and highlight all the for sale products. Don’t make them hesitate at the checkout page.